In [1]:
%cd '../..'

/home/tyz910/src/sberbank-covid19


In [2]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

countries = pd.read_csv('data/countries.csv')

In [3]:
URL = "https://en.wikipedia.org/wiki/National_responses_to_the_2019%E2%80%9320_coronavirus_pandemic"
page = requests.get(URL)
soup = BeautifulSoup(page.text)
tables = soup.findAll("table", {"class": "wikitable"})

In [4]:
data = []

def get_text(col):
    return re.sub(r"\[[^]]+\]", "", col.text.strip())

for row in tables[0].findAll("tr"):
    columns = row.findAll("td")
    if len(columns) < 4:
        continue

    if len(columns) == 5:
        country = get_text(columns[0])
        place = get_text(columns[1])
        columns = columns[2:]
    elif "colspan" in columns[0].attrs:
        country = get_text(columns[0])
        place = ""
        columns = columns[1:]
    else:
        place = get_text(columns[0])
        columns = columns[1:]
        
    start = get_text(columns[0])
    end = get_text(columns[1])
    level = get_text(columns[2])
    
    data.append([country, place, start, end, level])
    
df = pd.DataFrame(data, columns=["Country", "Place", "Start date", "End date", "Level"])
df = pd.merge(df, countries, left_on='Country', right_on='name', how='inner').groupby('country')[['Start date', 'Level']].first()
df = df.rename(columns={
    'Start date': 'date',
    'Level': 'level',
})

df.to_csv("data/quarantine.csv")
df

,date,level
country,,
ALB,2020-03-13,National
ARE,2020-03-26,National
ARG,2020-03-19,National
ARM,2020-03-24,National
AUS,2020-03-23,National
...,...,...
TUN,2020-03-22,National
UKR,2020-03-17,National
VEN,2020-03-17,National
